# cs 5644 final project experiment 2b: predicting individual company prices

In [1]:
import math
import pandas as pd
from IPython.display import display
from sklearn import preprocessing
from sklearn import linear_model, neighbors
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
import numpy as np

# read in data

In [2]:
df_index = pd.read_csv('sp500_index.csv')
df_stocks = pd.read_csv('sp500_stocks.csv')
df_companies = pd.read_csv('sp500_companies.csv')

### create single data frame

had to convert date columns to datetime objects to merge them for some reason

In [3]:
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], utc=True, errors='coerce')
df_stocks['Date'] = pd.DatetimeIndex(df_stocks['Date']).date

df_index['Date'] = pd.to_datetime(df_index['Date'], utc=True)
df_index['Date'] = pd.DatetimeIndex(df_index['Date']).date

In [4]:
df = df_companies.merge(df_stocks, on='Symbol', how='inner')

In [5]:
df = df.merge(df_index, on='Date')
df.head()

Exchange Symbol              Shortname               Longname  \
0      NMS   AAPL             Apple Inc.             Apple Inc.   
1      NMS   MSFT  Microsoft Corporation  Microsoft Corporation   
2      NMS  GOOGL          Alphabet Inc.          Alphabet Inc.   
3      NMS   GOOG          Alphabet Inc.          Alphabet Inc.   
4      NMS   AMZN       Amazon.com, Inc.       Amazon.com, Inc.   

                   Sector                        Industry  Currentprice  \
0              Technology            Consumer Electronics        140.42   
1              Technology         Software—Infrastructure        229.25   
2  Communication Services  Internet Content & Information         97.86   
3  Communication Services  Internet Content & Information         98.71   
4       Consumer Cyclical                 Internet Retail        113.67   

       Marketcap        Ebitda  Revenuegrowth  ...  \
0  2256661643264  1.295570e+11          0.019  ...   
1  1721204342784  9.798300e+10          0.124  ...   
2  1281926889472  9.688700e+10          0.126  ...   
3  1281502609408  9.688700e+10          0.126  ...   
4  1158024396800  5.262000e+10          0.072  ...   

                                 Longbusinesssummary    Weight        Date  \
0  Apple Inc. designs, manufactures, and markets ...  0.070093  2012-10-11   
1  Microsoft Corporation develops, licenses, and ...  0.053462  2012-10-11   
2  Alphabet Inc. provides various products and pl...  0.039818  2012-10-11   
3  Alphabet Inc. provides various products and pl...  0.039804  2012-10-11   
4  Amazon.com, Inc. engages in the retail sale of...  0.035969  2012-10-11   

   Adj Close      Close       High        Low       Open       Volume   S&P500  
0  19.372286  22.432142  23.114286  22.432142  23.089287  546081200.0  1432.84  
1  23.988226  28.950001  29.250000  28.870001  29.219999   41488500.0  1432.84  
2  18.805805  18.805805  18.981482  18.776026  18.841341   95260644.0  1432.84  
3  18.716845  18.716845  18.891689  18.687206  18.752213   95713418.0  1432.84  
4  12.211000  12.211000  12.465000  12.094500  12.400000   68946000.0  1432.84  

[5 rows x 24 columns]

# preprocessing

### drop irrelevant columns

In [6]:
df.drop(labels=['Shortname', 'Longname', 'City', 'Longbusinesssummary', 'Date'], axis=1, inplace=True)

### one hot encode

In [7]:
ohe = preprocessing.OneHotEncoder(dtype=int, sparse=False, handle_unknown="ignore")

In [8]:
data = ohe.fit_transform(df[['Exchange', 'Sector', 'Industry', 'State', 'Country']])

In [9]:
cats = pd.DataFrame(data, columns=ohe.get_feature_names())

/Users/caitlinhermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
snp_df = pd.concat([cats, df], axis=1)

In [11]:
snp_df.drop(columns=['Exchange', 'Sector', 'Industry', 'State', 'Country'], inplace=True)

In [12]:
snp_df

x0_BTS  x0_NGM  x0_NMS  x0_NYQ  x1_Basic Materials  \
0             0       0       1       0                   0   
1             0       0       1       0                   0   
2             0       0       1       0                   0   
3             0       0       1       0                   0   
4             0       0       1       0                   0   
...         ...     ...     ...     ...                 ...   
1244920       0       0       0       1                   0   
1244921       0       0       1       0                   0   
1244922       0       0       0       1                   0   
1244923       0       0       0       1                   0   
1244924       0       0       0       1                   0   

         x1_Communication Services  x1_Consumer Cyclical  \
0                                0                     0   
1                                0                     0   
2                                1                     0   
3                                1                     0   
4                                0                     1   
...                            ...                   ...   
1244920                          0                     0   
1244921                          0                     0   
1244922                          0                     0   
1244923                          0                     1   
1244924                          0                     0   

         x1_Consumer Defensive  x1_Energy  x1_Financial Services  ...  \
0                            0          0                      0  ...   
1                            0          0                      0  ...   
2                            0          0                      0  ...   
3                            0          0                      0  ...   
4                            0          0                      0  ...   
...                        ...        ...                    ...  ...   
1244920                      0          0                      0  ...   
1244921                      1          0                      0  ...   
1244922                      0          0                      0  ...   
1244923                      0          0                      0  ...   
1244924                      0          0                      0  ...   

         Revenuegrowth  Fulltimeemployees    Weight  Adj Close      Close  \
0                0.019           154000.0  0.070093  19.372286  22.432142   
1                0.124           221000.0  0.053462  23.988226  28.950001   
2                0.126           174014.0  0.039818  18.805805  18.805805   
3                0.126           174014.0  0.039804  18.716845  18.716845   
4                0.072          1523000.0  0.035969  12.211000  12.211000   
...                ...                ...       ...        ...        ...   
1244920          0.041             9300.0  0.000186  23.670000  23.670000   
1244921         -0.065            32000.0  0.000186  14.070000  14.070000   
1244922          0.741            22092.0  0.000162  41.180000  41.180000   
1244923        167.368            34700.0  0.000154  11.910000  11.910000   
1244924          0.163             3224.0  0.000127  21.250000  21.250000   

              High        Low       Open       Volume   S&P500  
0        23.114286  22.432142  23.089287  546081200.0  1432.84  
1        29.250000  28.870001  29.219999   41488500.0  1432.84  
2        18.981482  18.776026  18.841341   95260644.0  1432.84  
3        18.891689  18.687206  18.752213   95713418.0  1432.84  
4        12.465000  12.094500  12.400000   68946000.0  1432.84  
...            ...        ...        ...          ...      ...  
1244920  24.400000  23.650000  24.110001    1255555.0  3612.39  
1244921  14.210000  13.950000  14.020000    2551003.0  3612.39  
1244922  41.910000  40.884998  41.680000     844974.0  3612.39  
1244923  12.985000  11.500000  12.820000   28123604.0  3612.39  
1244924  21.49

### check number of missing values

In [13]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 250

In [14]:
percent_missing = snp_df.isnull().sum() * 100 / len(snp_df)
missing_value_df = pd.DataFrame({'column_name': snp_df.columns,
'percent_missing': percent_missing})

missing_value_df

column_name  \
x0_BTS                                                                        x0_BTS   
x0_NGM                                                                        x0_NGM   
x0_NMS                                                                        x0_NMS   
x0_NYQ                                                                        x0_NYQ   
x1_Basic Materials                                                x1_Basic Materials   
x1_Communication Services                                  x1_Communication Services   
x1_Consumer Cyclical                                            x1_Consumer Cyclical   
x1_Consumer Defensive                                          x1_Consumer Defensive   
x1_Energy                                                                  x1_Energy   
x1_Financial Services                                          x1_Financial Services   
x1_Healthcare                                                          x1_Healthcare   
x1_Industrials                                                        x1_Industrials   
x1_Real Estate                                                        x1_Real Estate   
x1_Technology                                                          x1_Technology   
x1_Utilities                                                            x1_Utilities   
x1_nan                                                                        x1_nan   
x2_Advertising Agencies                                      x2_Advertising Agencies   
x2_Aerospace & Defense                                        x2_Aerospace & Defense   
x2_Agricultural Inputs                                        x2_Agricultural Inputs   
x2_Airlines                                                              x2_Airlines   
x2_Apparel Manufacturing                                    x2_Apparel Manufacturing   
x2_Apparel Retail                                                  x2_Apparel Retail   
x2_Asset Management                                              x2_Asset Management   
x2_Auto & Truck Dealerships                              x2_Auto & Truck Dealerships   
x2_Auto Manufacturers                                          x2_Auto Manufacturers   
x2_Auto Parts                                                          x2_Auto Parts   
x2_Banks—Diversified                                            x2_Banks—Diversified   
x2_Banks—Regional                                                  x2_Banks—Regional   
x2_Beverages—Brewers                                            x2_Beverages—Brewers   
x2_Beverages—Non-Alcoholic                                x2_Beverages—Non-Alcoholic   
x2_Beverages—Wineries & Distilleries            x2_Beverages—Wineries & Distilleries   
x2_Biotechnology                                                    x2_Biotechnology   
x2_Building Materials                                          x2_Building Materials   
x2_Building Products & Equipment                    x2_Building Products & Equipment   
x2_Business Equipment & Supplies                    x2_Business Equipment & Supplies   
x2_Capital Markets                                                x2_Capital Markets   
x2_Chemicals                                                            x2_Chemicals   
x2_Communication Equipment                                x2_Communication Equipment   
x2_Computer Hardware                                            x2_Computer Hardware   
x2_Confectioners                                                    x2_Confectioners   
x2_Conglomerates                                                    x2_Conglomerates   
x2_Consulting Services                                        x2_Consulting Services   
x2_Consumer Electronics                                      x2_Consumer Electronics   
x2_Copper                                                                  x2_Copper   
x2_Credit Services                                                x2_Credit Services   
x2_Diagnostics & Research

### drop missing values

In [15]:
snp_df.dropna(inplace=True)
snp_df

x0_BTS  x0_NGM  x0_NMS  x0_NYQ  x1_Basic Materials  \
0             0       0       1       0                   0   
1             0       0       1       0                   0   
2             0       0       1       0                   0   
3             0       0       1       0                   0   
4             0       0       1       0                   0   
...         ...     ...     ...     ...                 ...   
1244920       0       0       0       1                   0   
1244921       0       0       1       0                   0   
1244922       0       0       0       1                   0   
1244923       0       0       0       1                   0   
1244924       0       0       0       1                   0   

         x1_Communication Services  x1_Consumer Cyclical  \
0                                0                     0   
1                                0                     0   
2                                1                     0   
3                                1                     0   
4                                0                     1   
...                            ...                   ...   
1244920                          0                     0   
1244921                          0                     0   
1244922                          0                     0   
1244923                          0                     1   
1244924                          0                     0   

         x1_Consumer Defensive  x1_Energy  x1_Financial Services  \
0                            0          0                      0   
1                            0          0                      0   
2                            0          0                      0   
3                            0          0                      0   
4                            0          0                      0   
...                        ...        ...                    ...   
1244920                      0          0                      0   
1244921                      1          0                      0   
1244922                      0          0                      0   
1244923                      0          0                      0   
1244924                      0          0                      0   

         x1_Healthcare  x1_Industrials  x1_Real Estate  x1_Technology  \
0                    0               0               0              1   
1                    0               0               0              1   
2                    0               0               0              0   
3                    0               0               0              0   
4                    0               0               0              0   
...                ...             ...             ...            ...   
1244920              1               0               0              0   
1244921              0               0               0              0   
1244922              0               1               0              0   
1244923              0               0               0              0   
1244924              0               0               1              0   

         x1_Utilities  x1_nan  x2_Advertising Agencies  \
0                   0       0                        0   
1                   0       0                        0   
2                   0       0                        0   
3                   0       0                        0   
4                   0       0                        0   
...               ...     ...                      ...   
1244920             0       0                        0   
1244921             0       0                        0   
1244922             0       0                        0   
1244923             0       0                        0   
1244924             0       0                        0   

         x2_Aerospace & Defense  x2_Agricultural Inputs  x2_Airlines  \
0                             0                       0            0

## separate x and y values
### + drop additional predictor columns

In [16]:
snp_xvalues = snp_df.drop(["Adj Close", "Close", "High", "Low", "Open", "Symbol"], axis=1)
snp_target = snp_df["Adj Close"]

# 

# 

# GOOG stock

In [48]:
goog_df = snp_df[snp_df['Symbol'] == 'GOOG']
goog_df

x0_BTS  x0_NGM  x0_NMS  x0_NYQ  x1_Basic Materials  \
3             0       0       1       0                   0   
498           0       0       1       0                   0   
993           0       0       1       0                   0   
1488          0       0       1       0                   0   
1983          0       0       1       0                   0   
...         ...     ...     ...     ...                 ...   
1242453       0       0       1       0                   0   
1242948       0       0       1       0                   0   
1243443       0       0       1       0                   0   
1243938       0       0       1       0                   0   
1244433       0       0       1       0                   0   

         x1_Communication Services  x1_Consumer Cyclical  \
3                                1                     0   
498                              1                     0   
993                              1                     0   
1488                             1                     0   
1983                             1                     0   
...                            ...                   ...   
1242453                          1                     0   
1242948                          1                     0   
1243443                          1                     0   
1243938                          1                     0   
1244433                          1                     0   

         x1_Consumer Defensive  x1_Energy  x1_Financial Services  \
3                            0          0                      0   
498                          0          0                      0   
993                          0          0                      0   
1488                         0          0                      0   
1983                         0          0                      0   
...                        ...        ...                    ...   
1242453                      0          0                      0   
1242948                      0          0                      0   
1243443                      0          0                      0   
1243938                      0          0                      0   
1244433                      0          0                      0   

         x1_Healthcare  x1_Industrials  x1_Real Estate  x1_Technology  \
3                    0               0               0              0   
498                  0               0               0              0   
993                  0               0               0              0   
1488                 0               0               0              0   
1983                 0               0               0              0   
...                ...             ...             ...            ...   
1242453              0               0               0              0   
1242948              0               0               0              0   
1243443              0               0               0              0   
1243938              0               0               0              0   
1244433              0               0               0              0   

         x1_Utilities  x1_nan  x2_Advertising Agencies  \
3                   0       0                        0   
498                 0       0                        0   
993                 0       0                        0   
1488                0       0                        0   
1983                0       0                        0   
...               ...     ...                      ...   
1242453             0       0                        0   
1242948             0       0                        0   
1243443             0       0                        0   
1243938             0       0                        0   
1244433             0       0                        0   

         x2_Aerospace & Defense  x2_Agricultural Inputs  x2_Airlines  \
3                             0                       0            0

In [18]:
googY = goog_df["Close"]
googX = goog_df.drop(["Close", "Symbol", "Adj Close", "High", "Low", "Open"], axis = 1)

## linear regression

In [19]:
regr = linear_model.LinearRegression()

In [20]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(goog_df):
    X_train, X_test = googX.iloc[train_index], googX.iloc[test_index]
    y_train, y_test = googY.iloc[train_index], googY.iloc[test_index]
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(googY[-25:].values)

r2=  0.975894816639439
root MSE= 5.516140749447938

r2=  0.9727682605121004
root MSE= 5.820466550885074

r2=  0.9704152441319159
root MSE= 6.094053553986678

r2=  0.9719574754618449
root MSE= 6.00066140943801

r2=  0.9696287021217
root MSE= 5.9491872112405435

average r2=  0.9721328997734
std=  0.0021821009255236943
average root MSE=  5.876101894999648
std=  0.20051273254534072

predicted:
[139.18984532 131.18359924 126.36803924 117.63343246 117.32235036
 120.9721682  124.98752285 110.01185544 106.95484645 116.68833721
 114.3499475  110.4008087   99.06779473 101.92484583 106.05311472
 101.87306898 109.96870376 112.64315451 114.49227452 120.64134749
 118.75740257 105.92149733  94.89800989  98.75460325  93.30995681]
ground truth:
[107.48000336 110.48000336 109.41999817 111.77999878 111.87000275
 105.30999756 105.87000275 103.90000153 103.62999725 103.84999847
 101.83000183 100.01000214 100.56999969  99.16999817  98.80999756
  98.08999634 100.73999786  98.08999634  96.15000153  99.3000030

## kNN regression 

### k=1

In [21]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=1, weights='uniform')

In [22]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(goog_df):
    X_train, X_test = googX.iloc[train_index], googX.iloc[test_index]
    y_train, y_test = googY.iloc[train_index], googY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(googY[-25:].values)

r2=  -0.4620537837099927
root MSE= 43.02496948519603

r2=  -0.3953985219332268
root MSE= 41.78657452038191

r2=  -0.44312099555308104
root MSE= 43.731341043649834

r2=  -0.5721465469624054
root MSE= 43.07845020177902

r2=  -0.42747003324500144
root MSE= 41.25143978789515

average r2=  -0.4600379762807415
std=  0.060162296014598944
average root MSE=  42.57455500778039
std=  0.9128509883212893

predicted:
[ 74.29650116  46.43899918  27.00635338  29.07517433  83.03549957
  41.76200104  31.1345005   36.90299988  31.28849983 115.66000366
  30.91250038  34.88850021  39.00400162  67.39150238  26.82335663
  47.67100143  28.68524551  42.28099823 122.65000153  62.79999924
  27.01732445  25.80926132  29.44914627  39.72800064 136.02850342]
ground truth:
[107.48000336 110.48000336 109.41999817 111.77999878 111.87000275
 105.30999756 105.87000275 103.90000153 103.62999725 103.84999847
 101.83000183 100.01000214 100.56999969  99.16999817  98.80999756
  98.08999634 100.73999786  98.08999634  96.150001

### k=5

In [23]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=5, weights='uniform')

In [24]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(goog_df):
    X_train, X_test = googX.iloc[train_index], googX.iloc[test_index]
    y_train, y_test = googY.iloc[train_index], googY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(googY[-25:].values)

r2=  0.11354995708438131
root MSE= 32.398883111265114

r2=  0.05613181918800103
root MSE= 33.81269057600488

r2=  0.12549102434400572
root MSE= 33.609516815696644

r2=  0.057941856890841414
root MSE= 34.45676626902999

r2=  0.011682619813216943
root MSE= 35.40183034016899

average r2=  0.07295945546408929
std=  0.041643013213992795
average root MSE=  33.93593742243313
std=  0.99039193119214

predicted:
[ 46.07760086  71.06159897 117.35139999  78.27180023  58.33261414
  54.98391533  39.5494545   85.7320015   93.56719894  55.7784996
  65.81781425  73.74597282  54.74573402  72.92969971  68.78736839
  76.8927002   90.58530197 104.3253006   44.77000008  92.52679977
  48.44914932  88.31273117  66.9659996   58.33261414  60.95799942]
ground truth:
[107.48000336 110.48000336 109.41999817 111.77999878 111.87000275
 105.30999756 105.87000275 103.90000153 103.62999725 103.84999847
 101.83000183 100.01000214 100.56999969  99.16999817  98.80999756
  98.08999634 100.73999786  98.08999634  96.15000153

### k=11

In [25]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=11, weights='uniform')

In [26]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(goog_df):
    X_train, X_test = googX.iloc[train_index], googX.iloc[test_index]
    y_train, y_test = googY.iloc[train_index], googY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(googY[-25:].values)

r2=  0.09245943480701824
root MSE= 31.884859444767038

r2=  0.19438873497539633
root MSE= 32.79668503907205

r2=  0.14537032199457878
root MSE= 31.877529441551644

r2=  0.17953205016916984
root MSE= 30.65888628383448

r2=  0.1679852160412042
root MSE= 34.25408402287386

average r2=  0.15594715159747347
std=  0.03555693259127996
average root MSE=  32.29440884641981
std=  1.1923976999506931

predicted:
[ 72.57113682  74.3557063   65.44350641  85.80002386  65.5863377
  79.65899259  55.97698784  55.90108958  39.74347531  50.03316741
  52.96548861  46.46080763  50.54111567  76.01688541  95.39979519
 103.53945333  50.91572553  69.04112018  62.95512546  75.04445405
  72.49002751  77.54156772  73.93568247  80.42118766 112.39995436]
ground truth:
[107.48000336 110.48000336 109.41999817 111.77999878 111.87000275
 105.30999756 105.87000275 103.90000153 103.62999725 103.84999847
 101.83000183 100.01000214 100.56999969  99.16999817  98.80999756
  98.08999634 100.73999786  98.08999634  96.15000153  

## neural network

In [27]:
X = googX.to_numpy()
y = googY.to_numpy()

### activation='relu'

layers=2, size=100

In [28]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(goog_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=2000, activation="relu", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9884 ±0.0012
RMSE = 3.7888 ±0.1209


layers=5, size=100

In [29]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(goog_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100), max_iter=2000, activation="relu", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9880 ±0.0010
RMSE = 3.8499 ±0.1118


### activation='tanh'

layers=2, size=100

In [30]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(goog_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=2000, activation="tanh", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.9891 ±0.0013
RMSE = 3.6719 ±0.1346


layers=5, size=100

In [31]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(goog_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100), max_iter=2000, activation="tanh", random_state=2)
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = -0.0031 ±0.0039
RMSE = 35.2655 ±1.2922


# 

# 

# TWTR stock 

In [49]:
twtr_df = snp_df[snp_df['Symbol'] == 'TWTR']
twtrY = twtr_df["Close"]
twtrX = twtr_df.drop(["Close", "Symbol", "Adj Close", "High", "Low", "Open"], axis = 1)

### linear regression

In [33]:
twtr_regr = linear_model.LinearRegression()

In [34]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
MSEscores = []

for train_index, test_index in kf.split(twtr_df):
    X_train, X_test = twtrX.iloc[train_index], twtrX.iloc[test_index]
    y_train, y_test = twtrY.iloc[train_index], twtrY.iloc[test_index]
    twtr_regr.fit(X_train, y_train)
    y_pred = twtr_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    MSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(MSEscores)))
print("std=  {}".format(np.std(MSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(twtrY[-25:].values)

r2=  0.22598677991173755
root MSE= 11.743360142060968

r2=  0.2677381790551955
root MSE= 12.3946871511972

r2=  0.23546553056478137
root MSE= 12.370368780108237

r2=  0.2523995676635654
root MSE= 12.35147358669518

r2=  0.23199363928595917
root MSE= 12.318548017693448

average r2=  0.2427167392962478
std=  0.015279306631564722
average root MSE=  12.235687535551005
std=  0.24741804579789875

predicted:
[50.73065602 49.99139571 47.3383969  48.29001235 47.34076958 47.0983322
 46.34390367 46.88275698 43.61226735 44.07417793 44.12281794 44.34161323
 45.00113443 44.31831009 45.31729447 45.84487754 47.16866531 47.63532126
 46.34237837 45.90614361 46.4245749  44.0736695  43.80301413 42.34237339
 43.12883314]
ground truth:
[38.65000153 41.20000076 41.84999847 42.18999863 41.40999985 41.74000168
 41.90000153 42.13999939 41.45000076 41.65999985 41.68000031 41.27000046
 41.40000153 41.58000183 41.52000046 42.09000015 43.25       42.74000168
 43.84000015 42.54000092 52.         51.29999924 49.38999

### kNN regression

### k=1

In [35]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=1, weights='uniform')

In [36]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(twtr_df):
    X_train, X_test = twtrX.iloc[train_index], twtrX.iloc[test_index]
    y_train, y_test = twtrY.iloc[train_index], twtrY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(twtrY[-25:].values)

r2=  -0.977846523105254
root MSE= 19.015128209934474

r2=  -0.8841643764745799
root MSE= 20.082020156342026

r2=  -0.9401524357338331
root MSE= 20.718752237447056

r2=  -1.0724910007555652
root MSE= 19.29002801364505

r2=  -0.8666495565519137
root MSE= 18.96283128270351

average r2=  -0.9482607785242291
std=  0.07367896569359832
average root MSE=  19.61375198001442
std=  0.6822237217484801

predicted:
[58.20000076 26.31999969 38.79000092 16.14999962 45.08000183 38.49000168
 15.77000046 21.68000031 30.29999924 19.54000092 18.5        25.62000084
 17.37000084 24.51000023 66.01000214 39.61000061 24.         24.01000023
 16.61000061 41.68000031 32.25999832 54.63999939 18.07999992 43.00999832
 49.13999939]
ground truth:
[38.65000153 41.20000076 41.84999847 42.18999863 41.40999985 41.74000168
 41.90000153 42.13999939 41.45000076 41.65999985 41.68000031 41.27000046
 41.40000153 41.58000183 41.52000046 42.09000015 43.25       42.74000168
 43.84000015 42.54000092 52.         51.29999924 49.3899

### k=5

In [37]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=5, weights='uniform')

In [38]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(twtr_df):
    X_train, X_test = twtrX.iloc[train_index], twtrX.iloc[test_index]
    y_train, y_test = twtrY.iloc[train_index], twtrY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(twtrY[-25:].values)

r2=  -0.12049367082048801
root MSE= 14.564687393624999

r2=  -0.16145264515627678
root MSE= 15.019447737643256

r2=  -0.10030366598015972
root MSE= 14.348536426084893

r2=  -0.16491944136478565
root MSE= 15.70930548781854

r2=  -0.15703857195993742
root MSE= 15.476952564335445

average r2=  -0.1408415990563295
std=  0.025784731312868927
average root MSE=  15.023785921901426
std=  0.5180828584654836

predicted:
[40.60000038 33.97400055 36.20200005 26.91600037 33.79799919 44.43199997
 24.91799984 39.90599976 34.52400055 48.47000046 41.10999985 54.63399963
 41.13600006 36.48199921 40.78600006 31.24799995 38.98199997 47.25000076
 31.62599907 42.58999939 38.80600052 42.1859993  44.06599922 36.20200005
 40.75599937]
ground truth:
[38.65000153 41.20000076 41.84999847 42.18999863 41.40999985 41.74000168
 41.90000153 42.13999939 41.45000076 41.65999985 41.68000031 41.27000046
 41.40000153 41.58000183 41.52000046 42.09000015 43.25       42.74000168
 43.84000015 42.54000092 52.         51.2999992

### k=11

In [39]:
knn_regr = neighbors.KNeighborsRegressor(n_neighbors=11, weights='uniform')

In [40]:
kf = model_selection.KFold(n_splits=5, shuffle=True)

r2scores = []
RMSEscores = []

for train_index, test_index in kf.split(twtr_df):
    X_train, X_test = twtrX.iloc[train_index], twtrX.iloc[test_index]
    y_train, y_test = twtrY.iloc[train_index], twtrY.iloc[test_index]
    knn_regr.fit(X_train, y_train)
    y_pred = knn_regr.predict(X_test)
    r2scores.append(metrics.r2_score(y_test, y_pred))
    RMSEscores.append(math.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print("r2=  {}".format(metrics.r2_score(y_test, y_pred)))
    print("root MSE= {}".format(math.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    print()

print("average r2=  {}".format(np.mean(r2scores)))
print("std=  {}".format(np.std(r2scores)))
print("average root MSE=  {}".format(np.mean(RMSEscores)))
print("std=  {}".format(np.std(RMSEscores)))
print()

print("predicted:")
print(y_pred[-25:])
print("ground truth:")
print(twtrY[-25:].values)

r2=  -0.10044026242609871
root MSE= 14.353834683069362

r2=  -0.06371751296826322
root MSE= 14.716434925682142

r2=  -0.059362982028052125
root MSE= 14.739867023825628

r2=  -0.08935341932007823
root MSE= 14.642696736820348

r2=  -0.08481664031110636
root MSE= 14.59829396450838

average r2=  -0.07953816341071973
std=  0.015610446722940098
average root MSE=  14.61022546678117
std=  0.13785525783933425

predicted:
[37.30181798 36.5990904  41.55545391 34.70727331 30.40545472 31.98363634
 42.30727265 40.61181849 32.14636343 41.00000035 43.4518176  34.17363704
 34.47272786 33.99090932 37.17636334 35.25090911 36.05636354 41.73454527
 49.99636355 34.57545471 28.31545466 35.82909125 36.15272695 35.76090925
 37.08545373]
ground truth:
[38.65000153 41.20000076 41.84999847 42.18999863 41.40999985 41.74000168
 41.90000153 42.13999939 41.45000076 41.65999985 41.68000031 41.27000046
 41.40000153 41.58000183 41.52000046 42.09000015 43.25       42.74000168
 43.84000015 42.54000092 52.         51.29999

## neural network

In [41]:
X = twtrX.to_numpy()
y = twtrY.to_numpy()

### activation='relu'

layers=2, size=100

In [42]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(twtr_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=2000, activation="relu", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.5448 ±0.0120
RMSE = 9.4928 ±0.2625


layers=5, size=100 

In [43]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(twtr_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100), max_iter=2000, activation="relu", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.5507 ±0.0423
RMSE = 9.4204 ±0.4993


### activation='tanh'

layers=2, size=100

In [44]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(twtr_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100), max_iter=2000, activation="tanh", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = 0.6122 ±0.0270
RMSE = 8.7514 ±0.2216


layers=5, size=100

In [45]:
kfold = model_selection.KFold(5, shuffle=True, random_state=2)

r2= []
RMSE = []

for train_idx, test_idx in kfold.split(twtr_df):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    mlp_regr = MLPRegressor(hidden_layer_sizes=(100, 100, 100, 100, 100), max_iter=2000, activation="tanh", random_state=2)
    X_scaler = preprocessing.MinMaxScaler()
    
    X_train = X_scaler.fit_transform(X_train)    
    
    mlp_regr.fit(X_train, y_train)
    
    X_test = X_scaler.transform(X_test)
    y_pred = mlp_regr.predict(X_test)
    
    r2 += [metrics.r2_score(y_test, y_pred)]
    RMSE += [math.sqrt(metrics.mean_squared_error(y_test, y_pred))]

print("r2    = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))
print("RMSE = {:.4f} ±{:.4f}".format(np.mean(RMSE), np.std(RMSE)))

r2    = -0.0011 ±0.0007
RMSE = 14.0771 ±0.3085
